# Jacob White Final Project

In [88]:
import pandas as pd
import numpy as np
import os
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', 20)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

from IPython.display import Image
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.datasets import make_classification;

/Users/jacobwhite/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Import data

In [31]:
df = pd.read_pickle('/Users/jacobwhite/development/data/metaTables_july/metaClaims_invMod.pickle')

### Exploratory Data Analysis

In [3]:
df.shape

(2522038, 306)

In [4]:
df.describe

<bound method NDFrame.describe of         accepted_date accident_type adjustment_reason  \
0                 NaT           NaN               NaN   
1                 NaT           NaN               NaN   
2                 NaT           NaN               NaN   
3                 NaT           NaN               NaN   
4                 NaT           NaN               NaN   
5                 NaT           NaN               NaN   
6                 NaT           NaN               NaN   
7                 NaT           NaN               NaN   
8                 NaT           NaN               NaN   
9                 NaT           NaN               NaN   
...               ...           ...               ...   
2522028           NaT           NaN               NaN   
2522029           NaT           NaN               NaN   
2522030           NaT           NaN               NaN   
2522031           NaT           NaN               NaN   
2522032           NaT           NaN               NaN 

In [6]:
df.isnull().sum()

accepted_date                    1704261
accident_type                    2516992
adjustment_reason                2429127
adjustment_reason_description    2465413
adjustment_reason_id             2465413
adjustment_type                  2522038
admission_date                   2483729
allowed_amount                         0
alt_procedure_code               2518056
amount_awp_ingredient_cost       1977613
                                  ...   
unit_type                         951394
update_type                      1969146
updated_by_base_user_id           299673
value_codes                       951295
workflow_queue_desc              1688523
workflow_queue_id                1688523
external_reportable                    0
metaclaim_id                           0
paid_date                         434777
has_corresponding_claim                0
Length: 306, dtype: int64

In [7]:
df.isnull().sum().sum()

383711056

In [8]:
df.dtypes

accepted_date                    datetime64[ns]
accident_type                            object
adjustment_reason                        object
adjustment_reason_description            object
adjustment_reason_id                     object
adjustment_type                         float64
admission_date                   datetime64[ns]
allowed_amount                          float64
alt_procedure_code                       object
amount_awp_ingredient_cost              float64
                                      ...      
unit_type                                object
update_type                              object
updated_by_base_user_id                 float64
value_codes                              object
workflow_queue_desc                      object
workflow_queue_id                        object
external_reportable                        bool
metaclaim_id                             object
paid_date                                object
has_corresponding_claim                 

In [9]:
df.head()

,accepted_date,accident_type,adjustment_reason,adjustment_reason_description,adjustment_reason_id,...,workflow_queue_id,external_reportable,metaclaim_id,paid_date,has_corresponding_claim
0,NaT,NaN,NaN,NaN,NaN,...,NaN,True,M_99036_99115,NaN,False
1,NaT,NaN,NaN,NaN,NaN,...,NaN,True,M_539318_623141,2016-09-30,True
2,NaT,NaN,NaN,NaN,NaN,...,NaN,True,M_534922_617617,2016-09-23,True
3,NaT,NaN,NaN,NaN,NaN,...,NaN,True,M_518445_592291,NaN,False
4,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589830,NaN,False


For this analysis - I am interested in what factors drive cost. Cost is defined in the column "Benefit Amount". This data set has 305 possible explanatory variables over more than 2 million rows. Benefit Amount will be the response variable for this analysis.

### Analysis

In [32]:
df[df['serviceYearMonth'] == '201606']

,accepted_date,accident_type,adjustment_reason,adjustment_reason_description,adjustment_reason_id,...,workflow_queue_id,external_reportable,metaclaim_id,paid_date,has_corresponding_claim
4,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589830,NaN,False
5,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589799,NaN,False
6,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589798,NaN,False
7,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589797,NaN,False
8,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589796,NaN,False
9,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589795,NaN,False
10,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589794,NaN,False
11,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589793,NaN,False
12,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589792,NaN,False
13,NaT,NaN,NaN,7,7,...,NaN,True,M_516064_589791,NaN,False


In [33]:
df.shape

(2522038, 306)

In [34]:
df = df[df['serviceYearMonth'] == '201606']

In [35]:
df.shape

(88910, 306)

In [36]:
df.groupby('person_id').benefit_amount.sum()

person_id
1025.0        106.09
1029.0       1110.23
1164.0        454.00
1165.0       1572.65
1169.0        320.86
1250.0          0.00
1253.0        440.18
1255.0        199.00
1258.0          0.00
1374.0          5.74
              ...   
247580.0    53293.11
248067.0      354.25
249034.0     1125.00
249103.0      966.58
249437.0      316.50
251552.0        0.00
264083.0        0.00
299459.0        0.00
333074.0        0.00
337604.0        0.00
Name: benefit_amount, Length: 12423, dtype: float64

In [79]:
df['total_spend'] = df.groupby('person_id').benefit_amount.transform('sum')

In [80]:
df.total_spend

4          9406.30
5          9406.30
6          9406.30
7          9406.30
8          9406.30
9          9406.30
10         9406.30
11         9406.30
12         9406.30
13         9406.30
            ...   
2521481     233.27
2521482     233.27
2521487     183.75
2521488     183.75
2521489     183.75
2521490     183.75
2521491     183.75
2521492     183.75
2521493     183.75
2521494     183.75
Name: total_spend, Length: 88910, dtype: float64

In [74]:
spend_by_person

,benefit_amount
4,9406.30
5,9406.30
6,9406.30
7,9406.30
8,9406.30
9,9406.30
10,9406.30
11,9406.30
12,9406.30
13,9406.30


### Bring in demographics information from Metapersons file

In [38]:
mp = pd.read_pickle('/Users/jacobwhite/development/data/metaTables_july/metaPersons.pickle')

In [39]:
mp.shape

(131618, 212)

In [40]:
mp.describe

<bound method NDFrame.describe of         sponsor_id sponsor_name  person_id date_of_birth gender     ...      \
0             1000     DataSafe       1020    1976-11-06      M     ...       
1             1000     DataSafe       1021    1981-01-01      M     ...       
2             1000     DataSafe       1021    1981-01-01      M     ...       
3             1000     DataSafe       1022    2014-10-09      F     ...       
4             1000     DataSafe       1022    2014-10-09      F     ...       
5             1000     DataSafe       1023    1982-12-02      F     ...       
6             1000     DataSafe       1023    1982-12-02      F     ...       
7             1000     DataSafe       1023    1982-12-02      F     ...       
8             1000     DataSafe       1024    1971-08-22      F     ...       
9             1000     DataSafe       1024    1971-08-22      F     ...       
...            ...          ...        ...           ...    ...     ...       
131608        1092

In [41]:
mp.isnull().sum()

sponsor_id                          0
sponsor_name                        0
person_id                           0
date_of_birth                       1
gender                              0
does_ssn_exist                      0
sponsorship_id                      0
hire_date                       31311
termination_date               118823
subsidiary                      60642
                                ...  
dental_is_ch                        0
vision_is_ch                        0
has_non_medical_ch_coverage         0
is_hra                              0
is_hsa                              0
is_fsa                              0
is_dcfsa                            0
is_any_fsa                          0
is_cobra                            0
region_code                         0
Length: 212, dtype: int64

In [42]:
test = mp[mp.plan_year == 2016]

In [43]:
test.shape

(42422, 212)

In [44]:
mp = mp[mp.plan_year == 2016]

In [46]:
mp.person_id

0          1020
1          1021
3          1022
6          1023
8          1024
10         1025
12         1026
14         1027
16         1028
18         1029
          ...  
93780    347371
93782    347387
93784    347388
93786    347389
93788    347573
93790    347574
93792    347575
93794    347576
93796    347597
93798    348911
Name: person_id, Length: 42422, dtype: int64

### Join demographics data onto financial data

In [81]:
person_finance = df[['person_id', 'total_spend']]

In [82]:
person_finance

,person_id,total_spend
4,241078.0,9406.30
5,241078.0,9406.30
6,241078.0,9406.30
7,241078.0,9406.30
8,241078.0,9406.30
9,241078.0,9406.30
10,241078.0,9406.30
11,241078.0,9406.30
12,241078.0,9406.30
13,241078.0,9406.30


In [83]:
data = mp.merge(person_finance, how = "inner")

In [84]:
data.columns

Index(['sponsor_id', 'sponsor_name', 'person_id', 'date_of_birth', 'gender',
       'does_ssn_exist', 'sponsorship_id', 'hire_date', 'termination_date',
       'subsidiary',
       ...
       'vision_is_ch', 'has_non_medical_ch_coverage', 'is_hra', 'is_hsa',
       'is_fsa', 'is_dcfsa', 'is_any_fsa', 'is_cobra', 'region_code',
       'total_spend'],
      dtype='object', length=213)

In [85]:
Y = data.total_spend

In [87]:
X = data[['sponsor_name', 'gender', 'msa_code']]

## Multivariable Logistic Regression